### LS/DR9 Photometry for Fuji (EDR)

This notebook briefly shows how to work with the LS/DR9 value-added catalogs for a small set of targets in the Fuji production of the DESI Early Data Release (EDR). To run this notebook be sure to use the DESI kernel or, if working in a terminal, to first do
```
source /global/cfs/cdirs/desi/software/desi_environment.sh master
```

John Moustakas  
Siena College  

In [1]:
import os
import numpy as np
import fitsio
from astropy.table import Table, vstack

#### Specify the location of the DESI/EDR redshift catalogs and the LS/DR9 VAC.

In [2]:
zcatdir = '/global/cfs/cdirs/desi/public/edr/spectro/redux/fuji/zcatalog'
vacdir = '/global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets'

#### Select a small number of LRG targets from the SV3 (One Percent Survey).

To select these targets, we apply the appropriate targeting bitmask and we also require `ZWARN==0` and `0.700<Z<0.701` (to keep the number of targets relatively small).

In [3]:
def select_targets():
    """Select a small number of targets."""
    from desitarget.sv3.sv3_targetmask import desi_mask

    zcatfile = os.path.join(zcatdir, 'ztile-sv3-dark-cumulative.fits')
    zcat = Table(fitsio.read(zcatfile, 'ZCATALOG'))
    I = ((zcat['SV3_DESI_TARGET'] & desi_mask.mask('LRG') != 0) * 
         (zcat['ZWARN'] == 0) * (zcat['Z'] > 0.8) * (zcat['Z'] < 0.801))
    zcat = zcat[I]
    _, uindx = np.unique(zcat['TARGETID'], return_index=True)
    zcat = zcat[uindx]
    print('Selected {} unique LRG targets from {}'.format(len(zcat), zcatfile))
    return zcat

In [4]:
zcat = select_targets()
zcat

Selected 281 unique LRG targets from /global/cfs/cdirs/desi/public/edr/spectro/redux/fuji/zcatalog/ztile-sv3-dark-cumulative.fits


TARGETID,LASTNIGHT,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF [10],NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,PLATE_RA,PLATE_DEC,TILEID,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MEAN_PSF_TO_FIBER_SPECFLUX,MEAN_FIBER_X,MEAN_FIBER_Y,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,ZCAT_NSPEC,ZCAT_PRIMARY
int64,int32,int32,float64,float64,int64,float64,float64,int64,str6,str20,int64,float64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,str3,float32,float32,int32,float64,int32,int16,str8,int32,int32,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,str1,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int32,int16,float32,int16,int16,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,bool
39627733994572203,20210409,20210409,0.8008523150738169,0.00010623420398979009,0,8330.310567066073,131.78979131595528 .. -0.6235943103877426,7924,GALAXY,,10,245.29847194254398,0,525,525,398,0,216.1099636885723,-2.222680906536108,0.0,0.0,2015.5,5400.0,65537,1,TGT,69.31266,-389.522,103200,0.5449253146314536,1,9010,2161m022,318272,2475,REX,0.05094936,0.71716267,1.7657319,7.131158,30.597456,17.261692,1037.5953,355.87097,20.247322,2.4731605,0.620663,0.3846019,0.9469314,3.8243163,0.3846019,0.9469314,3.8243163,0,1.0,0.4638818,0.0,0.0,0,,0.0,0.0,0.0,0.0,S,103200,9,65537,0,0,0,0,0,0,216.1099636885723,-2.222680906536108,219,2,1873.4424,1,1,0.001,0.001,-0.002,0.002,216.10996753907654,0.0,-2.2226736411777495,0.0,0.7708227,69.29,-389.398,155.5641,0.122202404,30.15684,128.48015,623.8672,225.67726,3.3008459,0.8737976,13726.112,31.24699,2500.7856,0.046476454,2832.9636,15828.749,9.271786,44.782993,2.1547296e-05,110.00949,4.0919995e-06,0.0,4687.266,2.897045e-05,22.18631,49.44073,13881.677,141.37868,2530.9424,128.52663,8144.0967,16054.427,34.75894,95.09752,1,True
39627734070070941,20210506,20210506,0.8006431732727989,0.00010050359723636207,0,7757.85095988214,89.26175932613182 .. 5.688041476352526,7888,GALAXY,,10,99.16401129215956,9,507,9507,4827,0,220.64806700598308,-2.1613287492738573,0.0,0.0,2015.5,5400.0,2162697,1,TGT,-215.04991,-334.27103,103200,0.7964147885999158,1,9010,2206m022,318290,3741,REX,0.05389881,0.321439,1.2395084,5.612989,30.200502,21.998972,711.42267,178.00143,72.4403,2.5460095,0.62926084,0.14500271,0.5591483,2.532047,0.14500271,0.5591483,2.532047,0,1.0,0.6016942,0.0

#### Define a couple convenience functions.

Note: The same target can be observed on different tiles *and* in different surveys (sometimes with very different targeting information), so you have to be very careful to ensure you're matching to the correct target.

In [5]:
def read_targetphot(zcat, survey=None):
    """Read the targeting photometry for an input catalog of objects.   

    """
    targetfile = os.path.join(vacdir, 'targetphot-{}-fuji.fits'.format(survey))
    info = fitsio.read(targetfile, 'TARGETPHOT', columns=['TARGETID', 'TILEID'])
    I = np.hstack([np.where((tileid == info['TILEID']) * (targetid == info['TARGETID']))[0]
                   for tileid, targetid in zip(zcat['TILEID'], zcat['TARGETID'])])
    targ = Table(fitsio.read(targetfile, rows=I))
    print('Read targeting photometry for {} objects from {}'.format(len(targ), targetfile))
    
    return targ   

In [6]:
def read_tractorphot(zcat, verbose=False):
    from glob import glob
    from desimodel.footprint import radec2pix

    tractorphotfiles = glob(os.path.join(vacdir, 'tractorphot', 'tractorphot-nside4-hp???-fuji.fits'))
    
    hdr = fitsio.read_header(tractorphotfiles[0], 'TRACTORPHOT')
    tractorphot_nside = hdr['FILENSID']

    pixels = radec2pix(tractorphot_nside, zcat['TARGET_RA'], zcat['TARGET_DEC'])
    phot = []
    for pixel in set(pixels):
        J = pixel == pixels
        photfile = os.path.join(vacdir, 'tractorphot', 'tractorphot-nside4-hp{:03d}-fuji.fits'.format(pixel))
        targetids = fitsio.read(photfile, columns='TARGETID')
        K = np.where(np.isin(targetids, zcat['TARGETID'][J]))[0]
        
        if verbose:
            print('Reading photometry for {} objects from {}'.format(len(K), photfile))
            
        _phot = fitsio.read(photfile, rows=K)
        phot.append(Table(_phot))
    phot = vstack(phot)

    # Is there a better way to sort here??
    srt = np.hstack([np.where(tid == phot['TARGETID'])[0] for tid in zcat['TARGETID']]) 
    phot = phot[srt]
    
    return phot

In [7]:
targ = read_targetphot(zcat, survey='sv3')
targ

Read targeting photometry for 281 objects from /global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets/targetphot-sv3-fuji.fits


RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,RA,RA_IVAR,DEC,DEC_IVAR,DCHISQ [5],EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,REF_EPOCH,WISEMASK_W1,WISEMASK_W2,MASKBITS,LC_FLUX_W1 [15],LC_FLUX_W2 [15],LC_FLUX_IVAR_W1 [15],LC_FLUX_IVAR_W2 [15],LC_NOBS_W1 [15],LC_NOBS_W2 [15],LC_MJD_W1 [15],LC_MJD_W2 [15],SHAPE_R,SHAPE_E1,SHAPE_E2,SHAPE_R_IVAR,SHAPE_E1_IVAR,SHAPE_E2_IVAR,SERSIC,SERSIC_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PMRA,PMRA_IVAR,PMDEC,PMDEC_IVAR,PHOTSYS,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SCND_TARGET,SV1_SCND_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SURVEY,PROGRAM,TILEID
int16,int32,str8,int32,str4,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32,float32,float32,float32,int16,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,bool,float32,float32,float32,float32,float32,float32,str1,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,str7,str6,int32
9010,348249,1503p030,4836,REX,150.45411284687506,631843850000.0,3.0488058544740104,638477530000.0,29645.268 .. 0.0,0.02384526,3.4412696,5.3395705,12.4126625,783.67737,338.74933,73.05513,0.93184686,0.95356435,0.9737542,0.0006890712,0.0014742292,0.0005458714,0.004150894,0.0061612152,0.004935254,0.9993914,0.9989895,0.99918425,4,4,3,1427.1433,507.1191,108.7135,856.92194,343.04874,58.20784,41.160877,31.305044,116.992195,3965.0261,2.228321,0.56851447,0.0006949095,9.140773e-06,0.9959671,0.99752134,0.99947083,0.99980015,0,0,0,2.3665323,3.6719782,8.536084,2.3665328,3.6719797,8.53609,0.0,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.24541378,0.0,0.0,31099.662,0.0,0.0,1.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,S,39627859727225572,0.15449295165086852,1,103200,9,27259,0,0,0,0,0,0,0,0,0,0,720931,0,0,0,0,0,0,sv3,dark,1
9010,341042,1486p017,2303,DEV,148.61809588527206,9639039000.0,1.8428981967842308,13464578000.0,877.918 .. 987.3342,0.026492864,0.28786415,1.4193782,6.931558,262.98862,82.24565,18.071072,0.9245721,0.9485433,0.9708829,0.31044272,0.2938828,0.2850678,0.057947572,0.06504024,0.063426726,0.9445755,0.94263494,0.9408217,2,2,2,1361.8193,407.0872,82.8088,739.9874,226.35313,48.969025,29.413242,14.024562,220.53477,945.79456,1.8555826,0.48860922,0.0005780306,8.391115e-06,0.9955203,0.9972465,0.9994121,0.999778,0,0,0,0.08798595,0.43383428,2.1186376,0.092564836,0.4525494

In [8]:
tractor = read_tractorphot(zcat, verbose=True)
tractor

Reading photometry for 1 objects from /global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets/tractorphot/tractorphot-nside4-hp099-fuji.fits
Reading photometry for 2 objects from /global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets/tractorphot/tractorphot-nside4-hp035-fuji.fits
Reading photometry for 66 objects from /global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets/tractorphot/tractorphot-nside4-hp101-fuji.fits
Reading photometry for 15 objects from /global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets/tractorphot/tractorphot-nside4-hp102-fuji.fits
Reading photometry for 28 objects from /global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets/tractorphot/tractorphot-nside4-hp036-fuji.fits
Reading photometry for 17 objects from /global/cfs/cdirs/desi/public/edr/vac/lsdr9-photometry/fuji/v1.0/observed-targets/tractorphot/tractorphot-nside4-hp040-fuji

RELEASE,BRICKID,BRICKNAME,OBJID,BRICK_PRIMARY,MASKBITS,FITBITS,TYPE,RA,DEC,RA_IVAR,DEC_IVAR,BX,BY,DCHISQ [5],EBV,MJD_MIN,MJD_MAX,REF_CAT,REF_ID,PMRA,PMDEC,PARALLAX,PMRA_IVAR,PMDEC_IVAR,PARALLAX_IVAR,REF_EPOCH,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_G_N_OBS,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_N_OBS,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_N_OBS,GAIA_PHOT_VARIABLE_FLAG,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_ASTROMETRIC_EXCESS_NOISE_SIG,GAIA_ASTROMETRIC_N_OBS_AL,GAIA_ASTROMETRIC_N_GOOD_OBS_AL,GAIA_ASTROMETRIC_WEIGHT_AL,GAIA_DUPLICATED_SOURCE,GAIA_A_G_VAL,GAIA_E_BP_MIN_RP_VAL,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,APFLUX_G [8],APFLUX_R [8],APFLUX_Z [8],APFLUX_RESID_G [8],APFLUX_RESID_R [8],APFLUX_RESID_Z [8],APFLUX_BLOBRESID_G [8],APFLUX_BLOBRESID_R [8],APFLUX_BLOBRESID_Z [8],APFLUX_IVAR_G [8],APFLUX_IVAR_R [8],APFLUX_IVAR_Z [8],APFLUX_MASKED_G [8],APFLUX_MASKED_R [8],APFLUX_MASKED_Z [8],APFLUX_W1 [5],APFLUX_W2 [5],APFLUX_W3 [5],APFLUX_W4 [5],APFLUX_RESID_W1 [5],APFLUX_RESID_W2 [5],APFLUX_RESID_W3 [5],APFLUX_RESID_W4 [5],APFLUX_IVAR_W1 [5],APFLUX_IVAR_W2 [5],APFLUX_IVAR_W3 [5],APFLUX_IVAR_W4 [5],MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,NOBS_G,NOBS_R,NOBS_Z,NOBS_W1,NOBS_W2,NOBS_W3,NOBS_W4,RCHISQ_G,RCHISQ_R,RCHISQ_Z,RCHISQ_W1,RCHISQ_W2,RCHISQ_W3,RCHISQ_W4,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACFLUX_W1,FRACFLUX_W2,FRACFLUX_W3,FRACFLUX_W4,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,ANYMASK_G,ANYMASK_R,ANYMASK_Z,ALLMASK_G,ALLMASK_R,ALLMASK_Z,WISEMASK_W1,WISEMASK_W2,PSFSIZE_G,PSFSIZE_R,PSFSIZE_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,NEA_G,NEA_R,NEA_Z,BLOB_NEA_G,BLOB_NEA_R,BLOB_NEA_Z,PSFDEPTH_W1,PSFDEPTH_W2,PSFDEPTH_W3,PSFDEPTH_W4,WISE_COADD_ID,WISE_X,WISE_Y,LC_FLUX_W1 [15],LC_FLUX_W2 [15],LC_FLUX_IVAR_W1 [15],LC_FLUX_IVAR_W2 [15],LC_NOBS_W1 [15],LC_NOBS_W2 [15],LC_FRACFLUX_W1 [15],LC_FRACFLUX_W2 [15],LC_RCHISQ_W1 [15],LC_RCHISQ_W2 [15],LC_MJD_W1 [15],LC_MJD_W2 [15],LC_EPOCH_INDEX_W1 [15],LC_EPOCH_INDEX_W2 [15],SERSIC,SERSIC_IVAR,SHAPE_R,SHAPE_R_IVAR,SHAPE_E1,SHAPE_E1_IVAR,SHAPE_E2,SHAPE_E2_IVAR,LS_ID,TARGETID
int16,int32,str8,int32,bool,int16,int16,str3,float64,float64,float32,float32,float32,float32,float32,float32,float64,float64,str2,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,int16,float32,float32,int16,bool,float32,float32,int16,int16,float32,bool,float32,float32,float32,float32,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,uint8,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str8,float32,float32,float32,float32,float32,float32,int16,int16,float32,float32,float32,float32,float64,float64,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,int64,int64
9010,318272,2161m022,2475,True,0,0,REX,216.1099636885723,-2.222680906536108,37184377000.0,3596162